In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from PIL import Image as PilImage
from io import BytesIO
from IPython.display import display, HTML

# Load the dataset
file_path = 'data_lagu_baru.csv'
data = pd.read_csv(file_path)

# Load the emotion detection model
emotion_model_path = 'MobileNet-Classification-7emotions-CNN.h5'
emotion_model = load_model(emotion_model_path)

# Preprocess the Data
data_cleaned = data.dropna()
encoder = OneHotEncoder(categories=[['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']])
emotion_encoded = encoder.fit_transform(data_cleaned[['Emotion']]).toarray()
scaler = StandardScaler()
popularity_scaled = scaler.fit_transform(data_cleaned[['Popularity']])
features = np.hstack([emotion_encoded, popularity_scaled])
target = data_cleaned[['Name', 'Artist', 'Album', 'Release Date']].values
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train the Nearest Neighbors model with n_neighbors=5
model = NearestNeighbors(n_neighbors=6)
model.fit(X_train, y_train)

# Function to load and preprocess the image
def load_and_preprocess_image(img_path):
    img = load_img(img_path, color_mode='rgb', target_size=(48, 48))
    img = img_to_array(img)
    img = img.astype('float32') / 255
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to detect emotion
def detect_emotion(emotion_model, img):
    predictions = emotion_model.predict(img)
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
    emotion = emotion_labels[np.argmax(predictions)]
    return emotion

# Function to get song recommendations
def get_recommendations(emotion, popularity, model, encoder, scaler):
    emotion_encoded = encoder.transform([[emotion]]).toarray()
    popularity_scaled = scaler.transform([[popularity]])
    input_features = np.hstack([emotion_encoded, popularity_scaled])
    distances, indices = model.kneighbors(input_features)
    recommendations = y_train[indices[0]]
    return recommendations

# Set up Spotify API credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='ce0abf674e2748cab17e27c590cde4e5', client_secret='cd56dc3ae283425f9f5368a1ee261bee'))

# Function to fetch track URL and cover art from Spotify
def fetch_spotify_data(song_name, artist_name):
    query = f"track:{song_name} artist:{artist_name}"
    results = sp.search(q=query, limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        cover_url = track['album']['images'][0]['url']
        track_url = track['external_urls']['spotify']
        return cover_url, track_url
    return None, None

# Main function to capture emotion and recommend songs
def recommend_songs(img_path):
    img = load_and_preprocess_image(img_path)
    detected_emotion = detect_emotion(emotion_model, img)
    print("Detected Emotion:", detected_emotion)
    recommendations = get_recommendations(detected_emotion, 80, model, encoder, scaler)
    
    # Create a DataFrame for the recommendations
    recommendations_df = pd.DataFrame(recommendations, columns=['Name', 'Artist', 'Album', 'ReleaseDate'])
    
    # Prepare HTML for clickable cover images in a 3x3 grid with text above the image
    html_content = '''
    <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px;">
    '''
    for index, row in recommendations_df.iterrows():
        cover_url, track_url = fetch_spotify_data(row['Name'], row['Artist'])
        text_content = f'{row["Name"]}<br/>{row["Artist"]}<br/>{row["Album"]}'
        if cover_url and track_url:
            img_tag = f'<img src="{cover_url}" style="width: 100%; height: auto; border: 1px solid #ddd;" />'
            link_tag = f'<a href="{track_url}" target="_blank">{img_tag}</a>'
            html_content += f'''
            <div style="text-align: center;">
                <div style="margin-bottom: 5px;">{text_content}</div>
                {link_tag}
            </div>
            '''
        else:
            html_content += f'''
            <div style="text-align: center;">
                <div style="margin-bottom: 5px;">{text_content}</div>
            </div>
            '''
    
    html_content += '</div>'
    
    # Display the HTML content
    display(HTML(html_content))

# Example usage:
img_path = 'alman.jpeg'  # Replace with the path to the uploaded image
recommend_songs(img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Detected Emotion: Neutral


C:\Users\Ihsan\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\Ihsan\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
